In [2]:
from pathlib import Path

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [3]:
URL = "https://benchmark-container-app.braveflower-04bf6ce2.uksouth.azurecontainerapps.io/"
DATA_DIR = Path("../resources")

In [4]:
client = BenchmarkClient(url=URL)

# Workouts

In [5]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df = df.rename(columns={"name": "routine_name"})
df.head()

,workoutroutine,date,exercise,weight,reps
0,Arm Block Chest,2024-10-28,Bench Press,72.5,8
1,Arm Block Chest,2024-10-28,Bench Press,72.5,5
2,Arm Block Chest,2024-10-28,Bench Press,60.0,11
3,Arm Block Chest,2024-10-28,Dips,0.0,10
4,Arm Block Chest,2024-10-28,Dips,0.0,10


In [6]:
latest_workout_date = client.workouts.get_latest_workout()["date"]
df = df[df["date"] > latest_workout_date]
df.head()

,workoutroutine,date,exercise,weight,reps
934,Arm Block Back,2024-11-23,Pull Ups Neutral Grip,10.0,4
935,Arm Block Back,2024-11-23,Pull Ups Neutral Grip,0.0,10
936,Arm Block Back,2024-11-23,Dumbbell Prone Rows,34.0,8
937,Arm Block Back,2024-11-23,Dumbbell Prone Rows,34.0,8
938,Arm Block Back,2024-11-23,Dumbbell Prone Rows,28.0,15


In [7]:
dates = df["date"].unique()
dates = [str(date.date()) for date in dates]

In [8]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs = exs.rename(columns={"id": "exercise_id", "name": "exercise_name"})
exs.head()

,exercise_name,exercise_id
0,bench press,106
1,lean-in cable row,107
2,dumbbell shoulder press neutral grip,108
3,lat pulldown neutral grip,109
4,rope hammer curls,110


In [9]:
routines = pd.DataFrame(client.workout_routines.get_workout_routines())
routines = routines.rename(columns={"name": "routine_name", "id": "routine_id"})
routines.head()

,routine_name,routine_id,routine_exercises
0,wave loading 321 lower 1,41,"[{'exercise': {'name': 'trap bar deadlift', 'i..."
1,wave loading 321 upper 1,42,"[{'exercise': {'name': 'bench press', 'id': 10..."
2,wave loading 321 lower 2,43,"[{'exercise': {'name': 'barbell back squats', ..."
3,wave loading 321 upper 2,44,"[{'exercise': {'name': 'military press', 'id':..."
4,wave loading 642 lower 1,45,"[{'exercise': {'name': 'trap bar deadlift', 'i..."


In [10]:
df["exercise_lower"] = df.exercise.str.lower()
df["workoutroutine_lower"] = df.workoutroutine.str.lower()
df.head()

,workoutroutine,date,exercise,weight,reps,exercise_lower,workoutroutine_lower
934,Arm Block Back,2024-11-23,Pull Ups Neutral Grip,10.0,4,pull ups neutral grip,arm block back
935,Arm Block Back,2024-11-23,Pull Ups Neutral Grip,0.0,10,pull ups neutral grip,arm block back
936,Arm Block Back,2024-11-23,Dumbbell Prone Rows,34.0,8,dumbbell prone rows,arm block back
937,Arm Block Back,2024-11-23,Dumbbell Prone Rows,34.0,8,dumbbell prone rows,arm block back
938,Arm Block Back,2024-11-23,Dumbbell Prone Rows,28.0,15,dumbbell prone rows,arm block back


In [11]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="exercise_name")
df_all = df_all.merge(right=routines, left_on="workoutroutine_lower", right_on="routine_name")[["date", "weight", "reps", "routine_id", "exercise_id"]]
df_all = df_all.set_index(["date", "routine_id", "exercise_id"])
df_all

weight  reps
date       routine_id exercise_id              
2024-11-23 53         120            10.0     4
                      120             0.0    10
                      122            34.0     8
                      122            34.0     8
                      122            28.0    15
                      143            25.0    10
                      143            25.0    10
                      143            22.5    15
                      153            55.0     8
                      153            55.0     9
                      153            45.0    15
                      138             9.0    20
                      138             9.0    17
                      138             9.0    15
                      145            24.0    30
                      145            24.0    40
2024-11-24 56         129           230.0    20
                      129           230.0    20
                      129           230.0    20
                      129           230.0   230
                      117            34.0    13
                      117            34.0    13
                      117            34.0    13
                      117            34.0    13
                      152            26.0    24
                      152            26.0    24
                      152            26.0    24
                      147            45.0    20
                      147            45.0    20
                      147            45.0    20
                      147            45.0    20
2024-11-25 52         106            72.5     7
                      106            72.5     6
                      106            60.0    13
                      141            12.5    15
                      141            12.5    15
                      141            12.5    11
                      135            70.0     8
                      135            55.0    10
                      111             7.5    20
                      111             7.5    13
                      111             7.5    10
                      113            20.0    20
                      113            20.0    15
                      113            20.0    13
                      144             0.0    17
                      144             0.0    13

In [13]:
for date in dates:
    all_workouts = []
    routines = df_all.loc[date].index.get_level_values(0).unique()
    for routine in routines:
        all_exercises = []
        exercises = df_all.loc[(date, routine)].index.unique()
        for exercise in exercises:
            sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
            data = {"id": exercise, "sets": sets}
            all_exercises.append(data)
        try:
            response = client.workouts.create_workout(date=str(date), exercises=all_exercises, workoutroutine_id=routine, user_id=2)
            print(f"Created Workout for date {date}")
        except Exception as e:
            print(e)

C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(da

Created Workout for date 2024-11-23


C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")


Created Workout for date 2024-11-24


C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
C:\Users\jorda\AppData\Local\Temp\ipykernel_10524\62323669.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  sets = df_all.loc[(da

Created Workout for date 2024-11-25
